#Libraries

In [ ]:
import numpy as np
import cv2
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time

#Functions

In [ ]:
'''
      changing camera view to bird's eye view
in:   src and dst matrix
out:  M & it invers transform

'''

def perspectiveParameters(src, dst):

    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)

    return M, Minv

In [ ]:
'''
      transforming a frame
in:   img, M matrix and size of image
out:  bird eye view

'''
def perspective_transform(img, M, imgShape=(1280,720)):

    return cv2.warpPerspective(img, M, imgShape)

In [ ]:
'''
      resize the image
in:   img, size
out:  resized image

'''
def changeSize(img, size):
    reimg = cv.resize(img, size)
    return reimg

In [ ]:
'''
      RGB color space to Lab & HLS
in:   RGB frane
out:  M & it invers transform

'''
def changeColorSpace(img):

    img_Lab = cv.cvtColor(img, cv.COLOR_BGR2Lab)
    img_HLS = cv.cvtColor(img, cv.COLOR_BGR2HLS)
  
    return img_Lab[:,:,2], img_HLS[:,:,1]

In [ ]:
'''
      color frame to binary frame
in:   frame, thresh (optional)
out:  binary

'''
def change2binary(img, thresh=None):

    img = cv.GaussianBlur(img,(11,11),0)

    if thresh == None:
        thresh = np.quantile(img, 0.99)

    ret,bin = cv.threshold(img,thresh,255,cv.THRESH_BINARY)
    
    return bin

In [ ]:
'''
      cobining yellow color and white color
in:   two binary image (yellow and white lane)
out:  one binary image

'''
def combineYellowWhite(yellow, white):

    return yellow+white

In [ ]:
'''
      finding two main line on the left and right of the road by sliding window
in:   binary image
out:  cordinate of lines and windows

'''
def sliding_window(img):

    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    midpoint = int(histogram.shape[0]//2)
    quarter_point = int(midpoint//2)
    leftx_base = np.argmax(histogram[quarter_point:midpoint]) + quarter_point
    rightx_base = np.argmax(histogram[midpoint:(midpoint+quarter_point)]) + midpoint
    nwindows = 30
    window_height = int(img.shape[0]/nwindows)
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    leftx_current = leftx_base
    rightx_current = rightx_base
    margin = 50
    minpix = 20

    left_lane_inds = []
    right_lane_inds = []
    rectangle_data = []

    for window in range(nwindows):
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        rectangle_data.append((win_y_low, win_y_high, win_xleft_low, win_xleft_high, win_xright_low, win_xright_high))

        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &
                          (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &
                           (nonzerox < win_xright_high)).nonzero()[0]

        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = int(np.mean(nonzerox[good_right_inds]))


    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)


    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    left_fit, right_fit = (None, None)

    if len(leftx) != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit = np.polyfit(righty, rightx, 2)
    
    visualization_data = (rectangle_data, histogram)
    
    return left_fit, right_fit, left_lane_inds, right_lane_inds, visualization_data



In [ ]:

'''
      Draw the detected lane over the input image.
in:   frame and it's binary, lines, inverse matrix of transform
out:  colored frame

'''
def draw_lane(original_img, binary_img, l_fit, r_fit, Minv):

    new_img = np.copy(original_img)
    if l_fit is None or r_fit is None:
        return original_img
    warp_zero = np.zeros_like(binary_img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    h,w = binary_img.shape
    ploty = np.linspace(0, h-1, num=h)
    left_fitx = l_fit[0]*ploty**2 + l_fit[1]*ploty + l_fit[2]
    right_fitx = r_fit[0]*ploty**2 + r_fit[1]*ploty + r_fit[2]
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(color_warp, np.int_([pts]), (0,150, 0))
    cv2.polylines(color_warp, np.int32([pts_left]), isClosed=False, color=(0,0,255), thickness=15)
    cv2.polylines(color_warp, np.int32([pts_right]), isClosed=False, color=(0,0,255), thickness=15)
    newwarp = cv2.warpPerspective(color_warp, Minv, (w, h))
    newwarp = cv2.resize(newwarp, (new_img.shape[1],new_img.shape[0]))
    result = cv2.addWeighted(new_img, 1, newwarp, 0.5, 0)
    return result

#Video to frames

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!ffmpeg -i /road.mp4 -vf fps=25 out%5d.png

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [ ]:
frames = []
from os import listdir
frames = [f for f in listdir('./') if 'out' in f]
frames.sort()

In [ ]:
images = []
for frame in frames:
    images.append(cv.imread(frame))

#Lane detection

In [ ]:
src = np.float32([[50, 340], [590, 340], [390, 225], [250, 225]])
dst = np.float32([[120, 360], [ 490, 360], [490,   1], [120,   1]])

In [ ]:
result = cv.VideoWriter('result.avi',cv.VideoWriter_fourcc(*'DIVX'), 25, (1280,720))

framedetected = []
st = time.time()
times = []
M, Minv = perspectiveParameters(src, dst)

size = (640, 360)

for image in images:
   
    img = changeSize(image,size)

    warped =  perspective_transform(img, M, size)

    yellow, white = changeColorSpace(warped)
    yellowWhite = combineYellowWhite(change2binary(yellow),change2binary(white))

    left_fit, right_fit, left_lane_inds, right_lane_inds, visualization_data = sliding_window(yellowWhite)

    framedetected.append( draw_lane(image, yellowWhite, left_fit, right_fit, Minv) )

et = time.time()

for frame in framedetected:
    result.write(frame.astype(np.uint8))

result.release()

In [ ]:
!rm *.png

In [ ]:
print('Average execution time for a frame:', round((et - st)/len(images),4), 'second')
print('It\'s equal to:', round(len(images)/(et - st),1), 'frame per second.')

Average execution time for a frame: 0.022 second
It's equal to: 45.4 frame per second.
